<a href="https://colab.research.google.com/github/ohjisu320/study_data_analytics/blob/main/docs/quests/EDAs/querys_ShoppingMallDeliveryWithDate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [4]:
df_shop = pd.read_csv('ShoppingMallDeliveryWithDate.csv')

In [6]:
df_shop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46749 entries, 0 to 46748
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   결제수단    46749 non-null  object 
 1   배송번호    46749 non-null  object 
 2   배송시작일   46749 non-null  object 
 3   배송완료일   46749 non-null  object 
 4   상품구매금액  46749 non-null  int64  
 5   상품번호    46661 non-null  float64
 6   수량      46749 non-null  int64  
 7   주문경로    46749 non-null  object 
 8   주문일     46749 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 3.2+ MB


### 1. 상품구매금액이 80,000원 이상인 주문들을 찾으세요.

In [14]:
df_shop['상품구매금액']

0         71450
1        141240
2         13910
3         81600
4         94710
          ...  
46744     52800
46745    112200
46746     36000
46747     73460
46748     81620
Name: 상품구매금액, Length: 46749, dtype: int64

In [13]:
condition = "상품구매금액 > 80000"
df_shop[['상품구매금액']].query(condition)

,상품구매금액
1,141240
3,81600
4,94710
6,82170
8,300960
...,...
46735,158400
46736,93000
46742,86240
46745,112200


### 2. 모바일웹을 통해 주문된 상품 중, 상품구매금액이 평균 이상인 주문들을 찾으세요.

In [15]:
df_shop[['주문경로']]

,주문경로
0,PC쇼핑몰
1,모바일웹
2,모바일웹
3,모바일웹
4,모바일웹
...,...
46744,모바일웹
46745,모바일웹
46746,네이버 페이
46747,네이버 페이


In [25]:
df_shop['상품구매금액'].mean().astype(int)

69368

In [36]:
condition_mobile = "주문경로 == '모바일웹'"
df_shop_mobile = df_shop[['주문경로', '상품구매금액']].query(condition_mobile)
df_shop_mobile

,주문경로,상품구매금액
1,모바일웹,141240
2,모바일웹,13910
3,모바일웹,81600
4,모바일웹,94710
5,모바일웹,75240
...,...,...
46742,모바일웹,86240
46743,모바일웹,0
46744,모바일웹,52800
46745,모바일웹,112200


In [37]:
mean = df_shop_mobile['상품구매금액'].mean().astype(int)
mean

68979

In [38]:
condition_over = f"상품구매금액 > {mean}"
df_shop_mobile = df_shop_mobile.query(condition_over)
df_shop_mobile

,주문경로,상품구매금액
1,모바일웹,141240
3,모바일웹,81600
4,모바일웹,94710
5,모바일웹,75240
21,모바일웹,115170
...,...,...
46730,모바일웹,73460
46736,모바일웹,93000
46742,모바일웹,86240
46745,모바일웹,112200


### 3. 2019년 1월 7일 이전에 배송이 시작된 주문들을 찾으세요.

In [44]:
df_shop[['배송시작일']][:2]

,배송시작일
0,2019-01-07 08:56:00
1,2019-01-03 09:30:00


In [43]:
def convert_time(x):
    date, ampm, time = x.split()
    if ampm == "오후" and time != "12:00":
        hour, minute = map(int, time.split(":"))
        time = f"{hour+12 if hour != 12 else hour}:{minute}"
    return pd.to_datetime(f"{date} {time}", format="%Y-%m-%d %H:%M")
pass
df_shop["배송시작일"] = df_shop["배송시작일"].apply(convert_time)

In [49]:
condition = "배송시작일 < 20190107"
df_shop.query(condition).sort_values(by='배송시작일')

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일
45119,무통장입금,D-20170101-0000015-00,2017-01-17 08:26:00,2017-01-17 오전 8:26,66500,45.0,1,모바일웹,20170101
45120,"쿠폰,카드",D-20170115-0000013-00,2017-01-19 08:56:00,2017-01-19 오후 7:51,17000,67.0,1,PC쇼핑몰,20170115
45121,"쿠폰,무통장입금",D-20170118-0000019-00,2017-01-19 19:55:00,2017-01-20 오후 4:14,73700,46.0,1,PC쇼핑몰,20170118
45122,"쿠폰,카드",D-20170123-0000021-00,2017-01-23 15:26:00,2017-01-24 오후 6:00,17000,67.0,1,PC쇼핑몰,20170123
45123,"쿠폰,카드",D-20170202-0000010-00,2017-02-02 17:54:00,2017-02-04 오후 3:26,72000,186.0,1,PC쇼핑몰,20170202
...,...,...,...,...,...,...,...,...,...
195,카드,D-20190102-0000938-03,2019-01-03 18:20:00,2019-01-13 오전 4:38,72270,378.0,1,네이버 페이,20190102
193,카드,D-20190102-0000938-01,2019-01-03 18:20:00,2019-01-13 오전 4:38,82170,380.0,1,네이버 페이,20190102
132,카드,D-20190102-0000463-01,2019-01-03 18:20:00,2019-01-28 오후 4:33,388080,1217.0,3,네이버 페이,20190102
134,카드,D-20190102-0000498-01,2019-01-03 18:20:00,2019-01-28 오후 4:15,129360,1217.0,1,네이버 페이,20190102


### 4. 배송 시작일과 배송 완료일이 3일 이상 차이 나는 주문들을 찾으세요.


In [51]:
def convert_time(x):
    date, ampm, time = x.split()
    if ampm == "오후" and time != "12:00":
        hour, minute = map(int, time.split(":"))
        time = f"{hour+12 if hour != 12 else hour}:{minute}"
    return pd.to_datetime(f"{date} {time}", format="%Y-%m-%d %H:%M")
pass
df_shop["배송완료일"] = df_shop["배송완료일"].apply(convert_time)

In [54]:
df_shop["배송기간"] = df_shop["배송완료일"]-df_shop["배송시작일"]
df_shop["배송기간"] = df_shop["배송기간"].dt.days

In [55]:
condition = '배송기간 >= 3 '
df_shop.query(condition)

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송기간
0,무통장입금,D-20181227-0000648-00,2019-01-07 08:56:00,2019-01-11 05:32:00,71450,1077.0,1,PC쇼핑몰,20181227,3
1,무통장입금,D-20181229-0000119-00,2019-01-03 09:30:00,2019-01-07 05:32:00,141240,42.0,2,모바일웹,20181229,3
2,무통장입금,D-20181230-0000100-00,2019-01-03 09:30:00,2019-01-07 05:32:00,13910,1271.0,1,모바일웹,20181230,3
3,무통장입금,D-20181231-0000087-00,2019-01-03 09:30:00,2019-01-07 05:32:00,81600,1085.0,1,모바일웹,20181231,3
4,무통장입금,D-20181231-0000108-00,2019-01-07 08:56:00,2019-01-11 05:32:00,94710,1106.0,1,모바일웹,20181231,3
...,...,...,...,...,...,...,...,...,...,...
46744,카드,D-20171229-0000036-00,2017-12-29 16:32:00,2018-01-02 05:31:00,52800,445.0,1,모바일웹,20171229,3
46745,카드,D-20171229-0000043-00,2018-01-03 09:45:00,2018-01-07 05:38:00,112200,172.0,2,모바일웹,20171229,3
46746,카드,D-20171230-0000022-01,2018-01-02 19:06:00,2018-01-11 02:46:00,36000,140.0,1,네이버 페이,20171230,8
46747,카드,D-20171231-0000017-01,2018-01-03 14:00:00,2018-01-06 18:06:00,73460,184.0,1,네이버 페이,20171231,3


### 5. 무통장입금을 결제수단으로 사용한 주문 중, 상품번호가 1000번 이상인 주문들을 찾으세요.


In [58]:
condition = '결제수단 == "무통장입금"'
df_shop_direct = df_shop.query(condition)
condition_over_1000 = '상품번호 >= 1000'
df_shop_direct.query(condition_over_1000).sort_values(by='상품번호')

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송기간
42681,무통장입금,D-20181213-0000162-00,2018-12-14 09:05:00,2018-12-18 05:32:00,39600,1017.0,2,PC쇼핑몰,20181213,3
41415,무통장입금,D-20181204-0000524-00,2018-12-04 20:41:00,2018-12-08 05:32:00,19800,1017.0,1,PC쇼핑몰,20181204,3
1135,무통장입금,D-20190111-0000391-00,2019-01-16 09:01:00,2019-01-20 05:32:00,24630,1019.0,1,PC쇼핑몰,20190111,3
26858,무통장입금,D-20180720-0000486-04,2018-07-24 09:50:00,2018-07-28 05:32:00,40210,1019.0,1,PC쇼핑몰,20180720,3
28003,무통장입금,D-20180801-0000379-01,2018-08-06 18:20:00,2018-08-10 05:32:00,60110,1022.0,1,PC쇼핑몰,20180801,3
...,...,...,...,...,...,...,...,...,...,...
12878,무통장입금,D-20190423-0000328-00,2019-04-30 18:33:00,2019-05-04 05:32:00,56130,1608.0,1,PC쇼핑몰,20190423,3
17546,무통장입금,D-20190602-0000017-04,2019-06-10 08:45:00,2019-06-14 05:32:00,90000,1625.0,2,모바일웹,20190602,3
15060,무통장입금,D-20190509-0000898-00,2019-05-10 19:05:00,2019-05-14 05:32:00,22500,1632.0,1,모바일웹,20190509,3
15664,무통장입금,D-20190515-0000778-00,2019-05-17 18:33:00,2019-05-21 05:32:00,6430,1633.0,1,PC쇼핑몰,20190515,3


### 6. 상품번호가 50번 이하이거나 1000번 이상인 주문들을 찾으세요.

In [69]:
condition_under_50 = '상품번호 <= 50'
condition_over_1000 = '상품번호 >=1000'
conditions = f'{condition_under_50} or {condition_over_1000}'
df_shop.query(conditions).sort_values(by='상품번호')

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송기간
7769,무통장입금,D-20190312-0000371-00,2019-03-12 18:38:00,2019-03-16 05:32:00,55870,38.0,1,모바일웹,20190312,3
5439,무통장입금,D-20190220-0001034-00,2019-02-22 09:03:00,2019-02-26 05:32:00,55870,38.0,1,PC쇼핑몰,20190220,3
18538,무통장입금,D-20180209-0000055-04,2018-02-09 17:09:00,2018-02-13 05:37:00,59400,38.0,1,PC쇼핑몰,20180209,3
44113,무통장입금,D-20181224-0000154-00,2018-12-28 18:22:00,2019-01-01 05:32:00,68970,38.0,1,PC쇼핑몰,20181224,3
10257,"적립금,무통장입금",D-20190402-0000221-01,2019-04-03 09:09:00,2019-04-07 05:32:00,55870,38.0,1,모바일웹,20190402,3
...,...,...,...,...,...,...,...,...,...,...
15664,무통장입금,D-20190515-0000778-00,2019-05-17 18:33:00,2019-05-21 05:32:00,6430,1633.0,1,PC쇼핑몰,20190515,3
16201,카드,D-20190520-0000631-00,2019-05-22 09:11:00,2019-05-26 05:32:00,6500,1679.0,1,모바일웹,20190520,3
16871,무통장입금,D-20190527-0000135-00,2019-05-29 11:31:00,2019-06-02 05:32:00,12740,1680.0,1,PC쇼핑몰,20190527,3
17250,"적립금,카드",D-20190529-0000343-00,2019-05-30 10:20:00,2019-06-03 05:32:00,55000,1681.0,1,모바일웹,20190529,3


### 7. 주문일이 20181231인 주문 중, 수량이 1개 이상인 주문들을 찾으세요.

In [65]:
df_shop['수량'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 46749 entries, 0 to 46748
Series name: 수량
Non-Null Count  Dtype
--------------  -----
46749 non-null  int64
dtypes: int64(1)
memory usage: 365.4 KB


In [68]:
condition_orderdate = '주문일 == 20181231'
condition_sku = '수량 >= 1'
conditions = f'{condition_orderdate} and {condition_sku}'
df_shop.query(conditions)

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송기간
3,무통장입금,D-20181231-0000087-00,2019-01-03 09:30:00,2019-01-07 05:32:00,81600,1085.0,1,모바일웹,20181231,3
4,무통장입금,D-20181231-0000108-00,2019-01-07 08:56:00,2019-01-11 05:32:00,94710,1106.0,1,모바일웹,20181231,3
5,무통장입금,D-20181231-0000108-01,2019-01-07 08:56:00,2019-01-11 05:32:00,75240,1374.0,1,모바일웹,20181231,3
6,무통장입금,D-20181231-0000184-00,2019-01-03 09:30:00,2019-01-07 05:32:00,82170,1071.0,1,PC쇼핑몰,20181231,3
7,무통장입금,D-20181231-0000184-01,2019-01-03 09:30:00,2019-01-07 05:32:00,63470,1373.0,1,PC쇼핑몰,20181231,3
...,...,...,...,...,...,...,...,...,...,...
45113,무통장입금,D-20181231-0000985-00,2019-01-03 09:30:00,2019-01-07 05:32:00,0,355.0,1,모바일웹,20181231,3
45114,카드,D-20181231-0001006-00,2019-01-03 09:30:00,2019-01-07 05:32:00,189420,1058.0,2,모바일웹,20181231,3
45115,카드,D-20181231-0001006-00,2019-01-03 09:30:00,2019-01-07 05:32:00,0,1243.0,1,모바일웹,20181231,3
45116,카드,D-20181231-0000595-04,2019-01-03 09:30:00,2019-01-07 05:32:00,147840,43.0,2,모바일웹,20181231,3


### 8. PC쇼핑몰을 통해 주문된 상품 중, 배송번호가 'D-20181231'로 시작하는 주문들을 찾으세요.

In [90]:
condition_pc = '주문경로 == "PC쇼핑몰"'
df_shop_pc = df_shop.query(condition_pc)
condition_number = df_shop_pc['배송번호'].str.contains('D-20181231')
df_shop_pc.loc[condition_number]

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송기간
6,무통장입금,D-20181231-0000184-00,2019-01-03 09:30:00,2019-01-07 05:32:00,82170,1071.0,1,PC쇼핑몰,20181231,3
7,무통장입금,D-20181231-0000184-01,2019-01-03 09:30:00,2019-01-07 05:32:00,63470,1373.0,1,PC쇼핑몰,20181231,3
8,무통장입금,D-20181231-0000184-02,2019-01-03 09:30:00,2019-01-07 05:32:00,300960,1374.0,4,PC쇼핑몰,20181231,3
9,무통장입금,D-20181231-0000184-03,2019-01-03 09:30:00,2019-01-07 05:32:00,121110,1037.0,1,PC쇼핑몰,20181231,3
10,무통장입금,D-20181231-0000205-00,2019-01-03 09:30:00,2019-01-07 05:32:00,94710,1058.0,1,PC쇼핑몰,20181231,3
...,...,...,...,...,...,...,...,...,...,...
45097,카드,D-20181231-0000822-00,2019-01-03 09:30:00,2019-01-07 05:32:00,0,1237.0,1,PC쇼핑몰,20181231,3
45102,카드,D-20181231-0000927-00,2019-01-03 09:30:00,2019-01-07 05:32:00,124250,90.0,1,PC쇼핑몰,20181231,3
45103,카드,D-20181231-0000927-00,2019-01-03 09:30:00,2019-01-07 05:32:00,0,584.0,1,PC쇼핑몰,20181231,3
45104,카드,D-20181231-0000953-00,2019-01-03 09:30:00,2019-01-07 05:32:00,71450,1077.0,1,PC쇼핑몰,20181231,3


### 9. 상품구매금액이 70,000원 이상 100,000원 이하인 주문들 중, 수량이 1개인 주문들을 찾으세요.

In [94]:
df_shop['수량']

0        1
1        2
2        1
3        1
4        1
        ..
46744    1
46745    2
46746    1
46747    1
46748    1
Name: 수량, Length: 46749, dtype: int64

In [96]:
condition_overunder = '70000 <= 상품구매금액 <= 100000'
condition_sku = '수량==1'
conditions = f"{condition_overunder} and {condition_sku}"
df_shop.query(conditions).sort_values(by='상품구매금액')

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송기간
17044,가상계좌,D-20190528-0000021-00,2019-05-29 11:31:00,2019-06-02 05:32:00,70000,397.0,1,모바일웹,20190528,3
18685,무통장입금,D-20180220-0000078-00,2018-02-21 18:13:00,2018-02-25 05:43:00,70000,602.0,1,PC쇼핑몰,20180220,3
23814,무통장입금,D-20180611-0000175-01,2018-06-12 09:03:00,2018-06-16 05:32:00,70000,1074.0,1,PC쇼핑몰,20180611,3
19295,카드,D-20180317-0000053-00,2018-03-19 17:35:00,2018-03-23 05:37:00,70000,602.0,1,모바일웹,20180317,3
18714,무통장입금,D-20180221-0000056-00,2018-02-21 18:13:00,2018-02-25 05:43:00,70000,602.0,1,PC쇼핑몰,20180221,3
...,...,...,...,...,...,...,...,...,...,...
26331,"쿠폰,카드",D-20180714-0000033-03,2018-07-16 18:36:00,2018-07-20 05:32:00,99700,82.0,1,모바일웹,20180714,3
23132,무통장입금,D-20180531-0000089-00,2018-06-01 15:12:00,2018-06-05 05:32:00,99700,82.0,1,PC쇼핑몰,20180531,3
30065,무통장입금,D-20180822-0000049-00,2018-08-22 18:44:00,2018-08-26 05:32:00,99700,82.0,1,PC쇼핑몰,20180822,3
21496,실시간계좌이체,D-20180508-0000421-01,2018-05-09 13:20:00,2018-05-19 03:12:00,99920,941.0,1,네이버 페이,20180508,9


### 10. 모바일웹을 통한 주문 중, 배송번호가 'D-20181230' 이후로 시작되고 상품구매금액이 50,000원 이상인 주문들을 찾으세요.

#### 배송번호의 앞이 주문일과 같은걸 이용했을 때

In [98]:
condition_mobile = '주문경로=="모바일웹"'
condition_over = '50000 <= 상품구매금액'
condition_orderdate = '주문일 >=20181230'
conditions = f"{condition_over} and {condition_orderdate} and {condition_mobile}"
df_shop.query(conditions).sort_values(by='주문일')

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송기간
44938,무통장입금,D-20181230-0000017-00,2018-12-31 18:17:00,2019-01-04 05:32:00,94710,1106.0,1,모바일웹,20181230,3
44976,"적립금,카드",D-20181230-0000236-03,2018-12-31 18:17:00,2019-01-04 05:32:00,476080,88.0,4,모바일웹,20181230,3
44975,"적립금,카드",D-20181230-0000236-02,2018-12-31 18:17:00,2019-01-04 05:32:00,116820,455.0,2,모바일웹,20181230,3
44970,무통장입금,D-20181230-0000224-00,2018-12-31 18:17:00,2019-01-04 05:32:00,95210,1322.0,1,모바일웹,20181230,3
44968,카드,D-20181230-0000191-00,2018-12-31 18:17:00,2019-01-04 05:32:00,75570,437.0,1,모바일웹,20181230,3
...,...,...,...,...,...,...,...,...,...,...
17543,무통장입금,D-20190602-0000017-01,2019-06-10 08:45:00,2019-06-14 05:32:00,52400,1373.0,1,모바일웹,20190602,3
17542,무통장입금,D-20190602-0000017-00,2019-06-10 08:45:00,2019-06-14 05:32:00,64070,1071.0,1,모바일웹,20190602,3
17545,카카오페이(카카오머니),D-20190604-0000018-00,2019-06-10 08:45:00,2019-06-14 05:32:00,61900,608.0,1,모바일웹,20190604,3
17548,페이나우(실시간 계좌이체),D-20190619-0000053-00,2019-06-20 17:32:00,2019-06-24 05:32:00,60900,1374.0,1,모바일웹,20190619,3


#### 배송번호를 분리해서 사용했을 때

In [113]:
df_shop['배송번호_date'] = df_shop['배송번호'].str.slice(2, 10)
df_shop['배송번호_date'] = df_shop['배송번호_date'].astype(int)
condition_orderdate = '배송번호_date >= 20181230'
condition_mobile = '주문경로=="모바일웹"'
condition_over = '50000 <= 상품구매금액'
conditions = f"{condition_over} and {condition_orderdate} and {condition_mobile}"
df_shop.query(conditions).sort_values(by='배송번호_date')

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송기간,배송번호_date
44938,무통장입금,D-20181230-0000017-00,2018-12-31 18:17:00,2019-01-04 05:32:00,94710,1106.0,1,모바일웹,20181230,3,20181230
44976,"적립금,카드",D-20181230-0000236-03,2018-12-31 18:17:00,2019-01-04 05:32:00,476080,88.0,4,모바일웹,20181230,3,20181230
44975,"적립금,카드",D-20181230-0000236-02,2018-12-31 18:17:00,2019-01-04 05:32:00,116820,455.0,2,모바일웹,20181230,3,20181230
44970,무통장입금,D-20181230-0000224-00,2018-12-31 18:17:00,2019-01-04 05:32:00,95210,1322.0,1,모바일웹,20181230,3,20181230
44968,카드,D-20181230-0000191-00,2018-12-31 18:17:00,2019-01-04 05:32:00,75570,437.0,1,모바일웹,20181230,3,20181230
...,...,...,...,...,...,...,...,...,...,...,...
17543,무통장입금,D-20190602-0000017-01,2019-06-10 08:45:00,2019-06-14 05:32:00,52400,1373.0,1,모바일웹,20190602,3,20190602
17542,무통장입금,D-20190602-0000017-00,2019-06-10 08:45:00,2019-06-14 05:32:00,64070,1071.0,1,모바일웹,20190602,3,20190602
17545,카카오페이(카카오머니),D-20190604-0000018-00,2019-06-10 08:45:00,2019-06-14 05:32:00,61900,608.0,1,모바일웹,20190604,3,20190604
17548,페이나우(실시간 계좌이체),D-20190619-0000053-00,2019-06-20 17:32:00,2019-06-24 05:32:00,60900,1374.0,1,모바일웹,20190619,3,20190619


In [107]:
df_shop['배송번호_date'].astype(int)

0        20181227
1        20181229
2        20181230
3        20181231
4        20181231
           ...   
46744    20171229
46745    20171229
46746    20171230
46747    20171231
46748    20171231
Name: 배송번호_date, Length: 46749, dtype: int64